In [ ]:
# Import all packages needed
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns
import json
from pathlib import Path
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [ ]:
# Import style.json
style_path: Path = Path.cwd().parent / 'style' / 'style.json'
with open(style_path, 'r') as f: # opens the style.json file
    style: dict = json.load(f) # loads as a dictionary object

In [ ]:
# Loading Data (data file must be in the same directory as jupyter notebook)
# Import data from excel file
# TODO: replace filename with the name of your data file
raw_data_filename: str ='au-038-d1-532NM.0.dpt'

# Tell program to read the data
raw_data: pd.DataFrame = pd.read_csv(raw_data_filename, sep='\t', header=None) # read data into a pandas dataframe by separating by "tabs" and there is no header

# Output data
raw_data

In [ ]:
# Data preprocessing specific to RAMAN
def preprocess(data: pd.DataFrame, normalize: bool = False, baseline: bool = False):
    """Function that applies transformation to the dataframe which will make it ready for plotting. Note, this is specific to UV-Vis."""
    # Rename column names
    data = data.rename(
        columns={
            data.columns[0]: "Wavenumber ($\mathregular{cm^{-1}}$)", # rename the first column to Wavenumber cm^-1
            data.columns[1]: "Intensity (a.u.)", # rename the second column to Intensity a.u.
        }
    )

    # Baseline correction
    if baseline: # if this is true, execute the following code
        col: str = data.columns[1] # get the column name of the Intensity by its column order in the dataframe
        data[col] = data[col] - np.average(data[col][0:50]) # subtract the average of the first 50 data points from the Intensity column

    # Normalize absorbance values
    if normalize: # if this is true, execute the following code
        col: str = data.columns[1] # get the column name of the Intensity by its column order in the dataframe
        max_val: float = data[col].max() # get the maximum value of the Intensity column
        data[col] = data[col] / max_val # divide the Intensity column by the maximum value of the Intensity column
        
    return data


# NOTE: change True to False if you don't want normalization or baseline correction
preprocessed_data: pd.DataFrame = preprocess(raw_data, False, False) # call the function we defined above
preprocessed_data

In [ ]:
# Plot the data!
# NOTE: User must change the sample names to the correct column names.
# seaborn styles
sns.set_style("ticks")
sns.set_context("notebook") # gives the blank white background theme
# sns.set_context("notebook", rc={'figure.figsize':(11.7,8.27)}) # sets the size of the figure

# sns.set_theme(context="notebook", style="white", font="Helvetica")

graph = sns.lineplot(
    data=preprocessed_data, # refers to dataframe that contains the data
    x="Wavenumber ($\mathregular{cm^{-1}}$)", # refers to column header to get all the X data
    y="Intensity (a.u.)", # refers to column header to get all the Y data
    palette="rocket_r",  #NOTE: User can change the palette here
    linewidth=style["graph"]["default"]["line_thickness"], # refers to the thickness of the line in the graph (this is set in the style.json file)
)
sns.despine() # this removes the top and right spines of the graph

graph.legend(
    fontsize=style["legend"]["font_size"], # refers to the font size of the legend (this is set in the style.json file)
    bbox_to_anchor=(0.7, 0.7), # refers to the position of the legend anchored in the figure
    title_fontsize=style["legend"]["title_font_size"], # refers to the font size of the legend title
    shadow=False, # refers to whether the legend has a shadow or not
    frameon=False, # refers to whether the legend has a frame or not
)
# font family
plt.rcParams['font.family'] = style["graph"]["default"]["font_family"] # refers to the font family of the text in the graph (this is set in the style.json file)

# y-axis and x-axis ticks
graph.xaxis.set_minor_locator(AutoMinorLocator(2)) # value of AutoMinorLocator dictates number of minor ticks between major ticks in X axis
graph.yaxis.set_minor_locator(AutoMinorLocator(2)) # value of AutoMinorLocator dictates number of minor ticks between major ticks in Y axis
graph.tick_params(axis="y", direction="in") # direction of major ticks in Y-axis
graph.tick_params(axis="x", direction="in") # direction of major ticks in X-axis
graph.tick_params(axis="y", which="minor", length=4, direction="in") # direction and length of minor ticks in Y-axis
graph.tick_params(axis="x", which="minor", length=4, direction="in") # direction and length of minor ticks in X-axis

# NOTE: User can change the x and y limits here
plt.xlim(1400, 2800)
# NOTE: User can change the y limits here
plt.ylim(-200, 1200)

# NOTE: make sure you comment this out if you want to save your figure in the directory. Otherwise, your figure will look like a blank screen
# plt.show()

# Save the figure
# NOTE: User can change the filename
plt.savefig("UV-Vis.svg", dpi=300, bbox_inches="tight")
plt.savefig("UV-Vis.jpg", dpi=300, bbox_inches="tight")